## Backtest

To see how our capital performs when trades are executed based on momentum scores and generated signals.

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt


Execute long (buy) and short (sell) positions on signal dates (first trading day of each month), close all positions on the last trading day of the month, and save the monthly performance results to data/backtest_results/monthly_performance.csv.

In [3]:
#Full year 2024

initial_capital = 1000

capital= initial_capital

momentum_path = Path("../data/momentum_data")
signals_path= Path("../data/signals")

signal_files = sorted(signals_path.glob("2024-*.csv"))

monthly_results=[]

for signal_file in signal_files:
    starting_capital= capital
    signal_date = pd.to_datetime(signal_file.stem)
    signals = pd.read_csv(signal_file, index_col="Symbol")
    long_tickers = signals[signals["signal"]=="long"].index.to_list()
    short_tickers = signals[signals["signal"]=="short"].index.to_list()
    all_tickers = long_tickers + short_tickers

    if not all_tickers:
        print(f"⚠️ No signals for {signal_date.strftime('%B %Y')}. Carrying forward capital.")
        continue

    allocation_per_stock = capital / len(all_tickers)
    portfolio={}

    #Get signal date from file name
    
    month_end_date = signal_date.replace(day=1) + pd.offsets.MonthEnd(0)

    #Entry(next trading day open)
    for ticker in all_tickers:
        file=momentum_path/f"{ticker}.csv"
        df=pd.read_csv(file, parse_dates=["Date"],index_col="Date")

        valid_date= df.index[df.index > signal_date]
        if valid_date.empty:
            continue

        entry_date = valid_date.min()
        open_price= df.loc[entry_date]["Open"]
        shares = allocation_per_stock/ open_price

        portfolio[ticker]={
            "position": "long" if ticker in long_tickers else "short",
            "price": open_price,
            "shares": shares
        }

    portfolio_value= 0
    long_profit=0
    short_profit=0

    for ticker,data in portfolio.items():
        file = momentum_path/ f"{ticker}.csv"
        df = pd.read_csv(file, parse_dates=["Date"],index_col="Date")

        available_dates = df.index[df.index <= month_end_date]
        if available_dates.empty:
            continue


        exit_date= available_dates.max()
        close_price = df.loc[exit_date]["Close"]

        if data["position"] == "long":
            pnl = (close_price - data["price"])* data["shares"]
            long_profit+= pnl
            portfolio_value+= allocation_per_stock+pnl

        else: #short
            pnl= (data["price"] - close_price)*data["shares"]
            short_profit+=pnl
            portfolio_value+= allocation_per_stock+pnl

    total_profit= long_profit + short_profit
    capital= portfolio_value

    print(f" {signal_date.strftime('%B %Y')}")
    print(f"  Long P&L:  ${long_profit:.2f}")
    print(f"  Short P&L: ${short_profit:.2f}")
    print(f"  Total P&L: ${total_profit:.2f}")
    print(f"  Portfolio Value: ${capital:.2f}")
    print("-" * 40)

    monthly_results.append({
        "Date": month_end_date,
        "Starting Capital": starting_capital,
        "portfolio_value": capital,
        "return_pct": (capital/starting_capital-1)%100,
        "long_profit":long_profit,
        "short_profit":short_profit,
        "total_profit":total_profit,
        "long_tickers":long_tickers,
        "short_tickers":short_tickers,

    })


# Final report
print(f"\n Final Portfolio Value: ${capital:.2f}")
print(f" Total Return: {((capital / initial_capital - 1) * 100):.2f}%")

results_df= pd.DataFrame(monthly_results)
results_df.to_csv("../data/backtest_results/monthly_performance.csv")


 January 2024
  Long P&L:  $36.97
  Short P&L: $31.07
  Total P&L: $68.04
  Portfolio Value: $1068.04
----------------------------------------
 February 2024
  Long P&L:  $54.55
  Short P&L: $-14.17
  Total P&L: $40.38
  Portfolio Value: $1108.42
----------------------------------------
 March 2024
  Long P&L:  $16.28
  Short P&L: $-34.24
  Total P&L: $-17.96
  Portfolio Value: $1090.45
----------------------------------------
 April 2024
  Long P&L:  $-24.36
  Short P&L: $22.70
  Total P&L: $-1.66
  Portfolio Value: $1088.79
----------------------------------------
 May 2024
  Long P&L:  $26.64
  Short P&L: $-11.22
  Total P&L: $15.42
  Portfolio Value: $1104.21
----------------------------------------
 June 2024
  Long P&L:  $-7.13
  Short P&L: $11.89
  Total P&L: $4.76
  Portfolio Value: $1108.97
----------------------------------------
 July 2024
  Long P&L:  $-16.00
  Short P&L: $-25.91
  Total P&L: $-41.91
  Portfolio Value: $1067.06
----------------------------------------
 Augu